# Extract UMAP embeddings for each plate of CellProfiler features

NOTE: We are using the feature selected profiles per plate.

In [1]:
import glob
import pathlib
import pandas as pd
import numpy as np
import umap

from pycytominer.cyto_utils import infer_cp_features

## Generate Embeddings

### Set constant for whole plates

In [2]:
# Set constants
umap_random_seed = 0
umap_n_components = 2

# Set embeddings directory
output_dir = pathlib.Path("results")
output_dir.mkdir(parents=True, exist_ok=True)

### Set paths to all plates

In [3]:
# Set input path with single-cell profiles
data_dir = pathlib.Path("../3.preprocessing_features/data/single_cell_profiles")

# Select only the feature selected files
file_suffix = "*sc_feature_selected.parquet"

# Obtain file paths for all feature selected plates
fs_files = glob.glob(f"{data_dir}/{file_suffix}")
print(f"There are {len(fs_files)} feature selected files with the following paths:")
fs_files

There are 4 feature selected files with the following paths:


['../3.preprocessing_features/data/single_cell_profiles/localhost240928120001_sc_feature_selected.parquet',
 '../3.preprocessing_features/data/single_cell_profiles/localhost240927060001_sc_feature_selected.parquet',
 '../3.preprocessing_features/data/single_cell_profiles/localhost240927120001_sc_feature_selected.parquet',
 '../3.preprocessing_features/data/single_cell_profiles/localhost240926150001_sc_feature_selected.parquet']

### Generate dictionary with plate and data

In [4]:
# Load feature data into a dictionary, keyed on plate name without the suffix
cp_dfs = {x.split("/")[-1].split("_sc")[0]: pd.read_parquet(x) for x in fs_files}

# Print out useful information about each dataset
print(cp_dfs.keys())
[cp_dfs[x].shape for x in cp_dfs]

dict_keys(['localhost240928120001', 'localhost240927060001', 'localhost240927120001', 'localhost240926150001'])


[(12745, 641), (12397, 652), (12902, 684), (16566, 657)]

### Fit UMAP for whole plates

In [5]:
# Initialize a dictionary to store UMAP embeddings for each plate
umap_results = {}

# Fit UMAP features per dataset and save
for plate in cp_dfs:
    # Set output file for the UMAP
    output_umap_file = pathlib.Path(output_dir, f"UMAP_{plate}.parquet")

    # Check if the output file already exists
    if output_umap_file.exists():
        print(f"Skipping {output_umap_file.stem} as it already exists.")
        continue

    # Make sure to reinitialize UMAP instance per plate
    umap_fit = umap.UMAP(
        random_state=umap_random_seed, n_components=umap_n_components, n_jobs=1
    )

    # Set dataframe as the current plate
    cp_df = cp_dfs[plate]

    # Process cp_df to separate features and metadata
    cp_features = infer_cp_features(cp_df)
    meta_features = infer_cp_features(cp_df, metadata=True)

    # Subset to only failing + DMSO and healthy + DMSO for fitting (avoid flooding from the compound treated cells)
    fit_subset = cp_df[
        (
            (cp_df["Metadata_cell_type"] == "healthy")
            & (cp_df["Metadata_treatment"] == "DMSO")
        )
        | (
            (cp_df["Metadata_cell_type"] == "failing")
            & (cp_df["Metadata_treatment"] == "DMSO")
        )
    ]

    # Fit on the subset
    umap_fit.fit(fit_subset.loc[:, cp_features])

    # Transform entire dataset and convert to pandas DataFrame
    embeddings = pd.DataFrame(
        umap_fit.transform(cp_df.loc[:, cp_features]),
        columns=[f"UMAP{x}" for x in range(0, umap_n_components)],
    )
    print(f"{embeddings.shape}: {plate}")

    # Combine with metadata
    cp_umap_with_metadata_df = pd.concat(
        [cp_df.loc[:, meta_features], embeddings], axis=1
    )

    # Add treatment type column
    cp_umap_with_metadata_df["Metadata_treatment_type"] = np.select(
        [
            (cp_umap_with_metadata_df["Metadata_cell_type"] == "healthy")
            & (cp_umap_with_metadata_df["Metadata_treatment"] == "DMSO"),
            (cp_umap_with_metadata_df["Metadata_cell_type"] == "failing")
            & (cp_umap_with_metadata_df["Metadata_treatment"] == "DMSO"),
            (cp_umap_with_metadata_df["Metadata_cell_type"] == "failing")
            & (cp_umap_with_metadata_df["Metadata_treatment"] != "DMSO"),
        ],
        ["healthy + DMSO", "failing + DMSO", "failing + compound"],
        default="other",
    )
    # Add new metadata column to the list
    meta_features.append("Metadata_treatment_type")

    # Check and adjust dtypes dynamically
    for col in cp_umap_with_metadata_df.columns:
        if col in meta_features:
            # Try converting to numeric first (if possible), if not, keep as string
            try:
                cp_umap_with_metadata_df[col] = pd.to_numeric(
                    cp_umap_with_metadata_df[col], errors="raise", downcast="integer"
                )
            except ValueError:
                # If can't convert to numeric, keep as string
                cp_umap_with_metadata_df[col] = cp_umap_with_metadata_df[col].astype(
                    str
                )
        else:
            # For UMAP embeddings, ensure they're float
            cp_umap_with_metadata_df[col] = cp_umap_with_metadata_df[col].astype(float)

        # Update the 'Pathway' column for failing + DMSO and healthy + DMSO
        cp_umap_with_metadata_df["Metadata_Pathway"] = cp_umap_with_metadata_df.apply(
            lambda row: (
                "failing + DMSO"
                if row["Metadata_cell_type"] == "failing"
                and row["Metadata_treatment"] == "DMSO"
                else (
                    "healthy + DMSO"
                    if row["Metadata_cell_type"] == "healthy"
                    and row["Metadata_treatment"] == "DMSO"
                    else row["Metadata_Pathway"]
                )
            ),
            axis=1,
        )

    # Store the UMAP result in the dictionary
    umap_results[plate] = cp_umap_with_metadata_df

    # Generate output file, drop unnamed column, and save
    cp_umap_with_metadata_df.to_parquet(output_umap_file, index=False)

(12745, 2): localhost240928120001
(12397, 2): localhost240927060001
(12902, 2): localhost240927120001
(16566, 2): localhost240926150001


## Combine all plates together to generate UMAP embeddings

In [6]:
# Get common features across all plates
common_columns = set.intersection(*[set(df.columns) for df in cp_dfs.values()])

# Use the first plate to identify metadata columns (those starting with 'Metadata_')
first_df = next(iter(cp_dfs.values()))
metadata_columns = [col for col in first_df.columns if col.startswith("Metadata_")]

# Get final columns to keep: intersection of common + metadata
final_columns = list(common_columns.union(metadata_columns))

# Subset each plate's dataframe to only those columns
cp_dfs_subset = {k: df[final_columns] for k, df in cp_dfs.items()}

# Combine all feature-selected plates with common features and metadata
combined_cp_df = pd.concat(cp_dfs_subset.values(), ignore_index=True)

# Verify shape and output
print(f"Combined shape: {combined_cp_df.shape}")
combined_cp_df.head()

Combined shape: (54610, 494)


,Cytoplasm_Texture_InfoMeas1_Hoechst_3_02_256,Nuclei_RadialDistribution_FracAtD_Mitochondria_4of4,Cytoplasm_RadialDistribution_FracAtD_Hoechst_4of4,Nuclei_Texture_Correlation_Hoechst_3_01_256,Cells_Texture_Correlation_Hoechst_3_03_256,Cells_AreaShape_Extent,Nuclei_AreaShape_Zernike_8_6,Nuclei_RadialDistribution_RadialCV_Hoechst_4of4,Nuclei_Texture_InfoMeas2_PM_3_03_256,Nuclei_Granularity_1_PM,...,Nuclei_AreaShape_Zernike_6_6,Metadata_treatment,Cytoplasm_Intensity_LowerQuartileIntensity_Mitochondria,Nuclei_Intensity_LowerQuartileIntensity_ER,Cytoplasm_Correlation_RWC_Actin_PM,Cytoplasm_Texture_InfoMeas1_PM_3_00_256,Nuclei_RadialDistribution_FracAtD_Mitochondria_1of4,Cytoplasm_RadialDistribution_RadialCV_PM_1of4,Nuclei_Texture_InfoMeas1_ER_3_02_256,Cells_Texture_Correlation_PM_3_00_256
0,0.264033,-2.036518,0.372932,-0.363018,1.025426,-1.749616,0.162665,-0.504237,0.734586,0.292277,...,-0.655811,DMSO,-0.049010,-0.398449,-0.549457,0.459913,3.861680,-0.402280,1.776024,0.263441
1,0.938108,-0.984578,1.663481,-1.349442,0.943382,-1.335279,1.199366,-0.123720,0.376406,-0.282644,...,0.387889,DMSO,0.937565,-0.156080,2.052919,-0.334728,-0.421215,-0.203213,-0.607593,0.598470
2,0.974481,-0.974071,2.040326,-0.197188,0.055931,-0.847134,0.011376,0.082748,0.322707,0.602630,...,0.477261,DMSO,-0.027293,-0.208989,-1.227606,-0.069321,-1.424283,-0.419449,0.184645,-0.749271
3,-0.523434,-1.015561,-0.799137,1.289177,-0.613698,-0.358182,0.413357,0.998536,0.763409,-1.258036,...,0.526755,DMSO,3.518795,-0.246336,2.540535,-1.336517,0.083470,-1.144538,-1.333053,1.094291
4,1.104776,-1.598902,-1.432393,1.480671,0.297621,0.574985,0.441018,0.960941,1.584880,-2.899216,...,0.336181,DMSO,4.736503,0.187826,2.025387,2.259020,5.840096,-0.752010,-0.668496,-0.573649


In [7]:
# Process combined_cp_df to separate features and metadata
combined_cp_features = infer_cp_features(combined_cp_df)
combined_meta_features = infer_cp_features(combined_cp_df, metadata=True)

# Subset to only failing + DMSO and healthy + DMSO for fitting
combined_fit_subset = combined_cp_df[
    (
        (combined_cp_df["Metadata_cell_type"] == "healthy")
        & (combined_cp_df["Metadata_treatment"] == "DMSO")
    )
    | (
        (combined_cp_df["Metadata_cell_type"] == "failing")
        & (combined_cp_df["Metadata_treatment"] == "DMSO")
    )
]

# Initialize UMAP instance
combined_umap_fit = umap.UMAP(
    random_state=umap_random_seed, n_components=umap_n_components, n_jobs=1
)

# Fit on the subset
combined_umap_fit.fit(combined_fit_subset.loc[:, combined_cp_features])

# Transform entire dataset and convert to pandas DataFrame
combined_embeddings = pd.DataFrame(
    combined_umap_fit.transform(combined_cp_df.loc[:, combined_cp_features]),
    columns=[f"UMAP{x}" for x in range(0, umap_n_components)],
)

# Combine with metadata
combined_cp_umap_with_metadata_df = pd.concat(
    [combined_cp_df.loc[:, combined_meta_features], combined_embeddings], axis=1
)

# Add treatment type column
combined_cp_umap_with_metadata_df["Metadata_treatment_type"] = np.select(
    [
        (combined_cp_umap_with_metadata_df["Metadata_cell_type"] == "healthy")
        & (combined_cp_umap_with_metadata_df["Metadata_treatment"] == "DMSO"),
        (combined_cp_umap_with_metadata_df["Metadata_cell_type"] == "failing")
        & (combined_cp_umap_with_metadata_df["Metadata_treatment"] == "DMSO"),
        (combined_cp_umap_with_metadata_df["Metadata_cell_type"] == "failing")
        & (combined_cp_umap_with_metadata_df["Metadata_treatment"] != "DMSO"),
    ],
    ["healthy + DMSO", "failing + DMSO", "failing + compound"],
    default="other",
)

# Update the 'Pathway' column for failing + DMSO and healthy + DMSO
combined_cp_umap_with_metadata_df["Metadata_Pathway"] = combined_cp_umap_with_metadata_df.apply(
    lambda row: (
        "failing + DMSO"
        if row["Metadata_cell_type"] == "failing"
        and row["Metadata_treatment"] == "DMSO"
        else (
            "healthy + DMSO"
            if row["Metadata_cell_type"] == "healthy"
            and row["Metadata_treatment"] == "DMSO"
            else row["Metadata_Pathway"]
        )
    ),
    axis=1,
)

# Save the combined UMAP embeddings to a file
combined_output_umap_file = pathlib.Path(output_dir, "UMAP_combined.parquet")
combined_cp_umap_with_metadata_df.to_parquet(combined_output_umap_file, index=False)

print(f"Combined UMAP embeddings saved to {combined_output_umap_file}")

Combined UMAP embeddings saved to results/UMAP_combined.parquet


In [8]:
combined_cp_umap_with_metadata_df.head()

,Metadata_ImageNumber,Metadata_Cells_Number_Object_Number,Metadata_WellRow,Metadata_Well,Metadata_Image_Count_Cells,Metadata_Nuclei_Number_Object_Number,Metadata_heart_failure_type,Metadata_Site,Metadata_Nuclei_Location_Center_Y,Metadata_WellCol,...,Metadata_Cytoplasm_Parent_Cells,Metadata_Plate,Metadata_Nuclei_Location_Center_X,Metadata_Cells_Location_Center_X,Metadata_cell_type,Metadata_Pathway,Metadata_treatment,UMAP0,UMAP1,Metadata_treatment_type
0,2,1,B,B02,9,6,None,f04,277.977481,2,...,1,localhost240928120001,855.599578,809.932735,healthy,healthy + DMSO,DMSO,14.966546,-0.237015,healthy + DMSO
1,3,1,B,B02,4,3,None,f10,209.544343,2,...,1,localhost240928120001,613.937003,640.790427,healthy,healthy + DMSO,DMSO,13.213906,0.074713,healthy + DMSO
2,5,1,B,B02,9,3,None,f14,280.023188,2,...,1,localhost240928120001,916.379710,927.500461,healthy,healthy + DMSO,DMSO,13.969444,0.775292,healthy + DMSO
3,7,1,B,B02,18,3,None,f18,80.037145,2,...,1,localhost240928120001,530.411492,515.843583,healthy,healthy + DMSO,DMSO,12.739195,0.869190,healthy + DMSO
4,1,1,B,B02,10,4,None,f00,69.265085,2,...,1,localhost240928120001,958.301695,955.244723,healthy,healthy + DMSO,DMSO,11.929558,0.025988,healthy + DMSO
